In [69]:
from nltk.corpus import stopwords
import pandas as pd
import re 

In [70]:
allstopwords = set(stopwords.words('english'))


In [71]:
with open('labeledData.txt', 'r', encoding='utf-8') as file:
    # Read each line from the file and strip newline characters
    descriptions = [line.strip() for line in file.readlines()]

In [72]:
descriptions = descriptions[1:]
descriptions

['"Last week I had the pleasure to NFT Paris by the hand of Arianee . Witnessing the transformative power of Web3 and NFTs in reshaping the concept of ownership, particularly in relation to our personal data, left me inspired and eager to delve deeper into the evolving landscape of decentralized technologies. The event sparked valuable insights, and I\'m excited to continue navigating the dynamic realm of blockchain innovations. #NFTParis #Web3 #OwnershipRevolution",Positive',
 '"Hello everyone Today, my friends Asma Ghamacha , Hermes Yan NTJAM NDJENG , Harold Geumtcheng , Aloys Aymrick Nzooh , Bryan Fozame and I had the chance to be part of the NFT Paris conference thanks to our school aivancity School for Technology, Business & Society Paris-Cachan where we learned a wealth of new information about blockchain, metaverse, web3 and its use cases across various industries such as finance, gaming, luxury, and more. During this enI had the privilege to engage in discussions with numerous 

In [73]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
eliminarAcentos = lambda str: str.translate(str.maketrans("áàäéèëíìïòóöùúüÀÁÄÈÉËÌÍÏÒÓÖÙÚÜ", "aaaeeeiiiooouuuAAAEEEIIIOOOUUU"))

corpus = [] 
labels = []
for i in descriptions:
    item = eliminarAcentos(i)
    item = item.lower()
    # 1. Quitamos todo lo que no sea una letra:
    item = re.sub("[^a-zA-Z]", " ", item) 
    
    # 2.1 Quitamos espacios multiples y dejamos solo uno:
    item = re.sub("[\s]+", " ", item)

    item = item.strip()

    # 3. Separamos oraciones para obtener puras palabras
    item = item.split()
    
    if len(item)> 1:
        labels.append(item[-1])
    
    item = item[0:len(item)-1]

    # 4.1 Además queremos quitar palabras irrelevantes como artículos
   
    item = [word for word in item if word not in allstopwords]
    

    # 4.2 Unimos el Stemmer/Lematization para cada palabra que no esté dentro de la lista de stopwords
    
    item = [lemmatizer.lemmatize(word) for word in item ]
    
    

    # 5. Unir oraciones con espacios entre cada palabra
    item = " ".join(item)
    
    if item != "":
        corpus.append(item)

In [74]:
corpus

['last week pleasure nft paris hand arianee witnessing transformative power web nfts reshaping concept ownership particularly relation personal data left inspired eager delve deeper evolving landscape decentralized technology event sparked valuable insight excited continue navigating dynamic realm blockchain innovation nftparis web ownershiprevolution',
 'hello everyone today friend asma ghamacha hermes yan ntjam ndjeng harold geumtcheng aloys aymrick nzooh bryan fozame chance part nft paris conference thanks school aivancity school technology business society paris cachan learned wealth new information blockchain metaverse web use case across various industry finance gaming luxury eni privilege engage discussion numerous brilliant web developer ceo company like maxence perray nomiks victor briere arianee ubisoft louis vuitton many others conversation provided deeper insight innovative technology intersection data science particularly term transparency tokenization blockchain datascien

In [75]:
labels

['positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral']

# Data augmentation
I am going to generate new data for neutral and negative labels, in order to have a more balanced set and to have more data in general. Sinc the crawler is very small and sometimes it makes me lose data.

In [76]:
import nltk
from nltk.corpus import wordnet
import random
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\matts\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\matts\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [77]:
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return set(synonyms)



def synonym_replacement(sentence, n):
    words = nltk.word_tokenize(sentence)
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:  # replace up to n words
            break

    sentence = ' '.join(new_words)
    sentence = re.sub("[^a-zA-Z]", " ", sentence) 
    return sentence

synonym_replacement("I am extremely exahusted today", len("i am extemely exahusted today")/2)

'ane Master of Arts exceedingly exahusted nowadays'

In [78]:
print(len(labels), len(corpus))

53 53


In [79]:
newData = []
newLabels = []
for i in range(2):
    for i in range(len(corpus)):
        sen = corpus[i]
        sen = synonym_replacement(sen, len(sen)//2 )
        newData.append(sen)
        newLabels.append(labels[i])

In [80]:
corpus = corpus+newData
labels = labels+newLabels
print(len(labels), len(corpus))

159 159


In [88]:
df = pd.DataFrame(
    {'text': corpus,
    'tone': labels}
)
df

,text,tone
0,last week pleasure nft paris hand arianee witn...,positive
1,hello everyone today friend asma ghamacha herm...,positive
2,digital world continues intersect traditional ...,neutral
3,wait welcome everyone paris week nft paris sho...,positive
4,dualmint set launch toji nft japanese sake tok...,positive
...,...,...
154,nft marketplace evolve provide lineament just ...,neutral
155,interpretation sound deduction nfts mightiness...,neutral
156,detailed analysis flow land nfts graphics eart...,neutral
157,nfts take up apply raw unexpected manner keepi...,neutral


In [89]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
df['tone'] = enc.fit_transform(df['tone'])
df

,text,tone
0,last week pleasure nft paris hand arianee witn...,2
1,hello everyone today friend asma ghamacha herm...,2
2,digital world continues intersect traditional ...,1
3,wait welcome everyone paris week nft paris sho...,2
4,dualmint set launch toji nft japanese sake tok...,2
...,...,...
154,nft marketplace evolve provide lineament just ...,1
155,interpretation sound deduction nfts mightiness...,1
156,detailed analysis flow land nfts graphics eart...,1
157,nfts take up apply raw unexpected manner keepi...,1


In [90]:
df['tone'].value_counts()

2    81
1    45
0    33
Name: tone, dtype: int64

# Prepare data for the model

In [95]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text, df.tone, test_size=0.2, random_state=42)

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
vectorizer = TfidfVectorizer()
transformed_X_train = vectorizer.fit_transform(X_train)
transformed_X_train = transformed_X_train.toarray()
y_train = y_train.fillna(0)
transformed_y_train = np.array(y_train)

In [97]:
transformed_X_test = vectorizer.transform(X_test)
y_test = y_test.fillna(0)
transformed_y_test = np.array(y_test)

# Training models

# 1. SVM

In [110]:
from sklearn.linear_model import SGDClassifier

sgd =  SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd.fit(transformed_X_train, transformed_y_train)
yPred = sgd.predict(transformed_X_test)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print('accuracy %s' % accuracy_score(yPred, transformed_y_test))
print(classification_report(transformed_y_test, yPred))

accuracy 0.875
              precision    recall  f1-score   support

           0       0.60      1.00      0.75         6
           1       1.00      0.70      0.82        10
           2       1.00      0.94      0.97        16

    accuracy                           0.88        32
   macro avg       0.87      0.88      0.85        32
weighted avg       0.93      0.88      0.88        32



# 2. Naive Bayes

In [113]:
from sklearn.naive_bayes import MultinomialNB

nb =  MultinomialNB()
nb.fit(transformed_X_train, transformed_y_train)
yPred = nb.predict(transformed_X_test)
print('accuracy %s' % accuracy_score(yPred, transformed_y_test))
print(classification_report(transformed_y_test, yPred))

accuracy 0.65625
              precision    recall  f1-score   support

           0       1.00      0.17      0.29         6
           1       1.00      0.40      0.57        10
           2       0.59      1.00      0.74        16

    accuracy                           0.66        32
   macro avg       0.86      0.52      0.53        32
weighted avg       0.80      0.66      0.60        32



In [114]:
from sklearn.linear_model import LogisticRegression

lr =  LogisticRegression(max_iter=5, n_jobs=3)
lr.fit(transformed_X_train, transformed_y_train)
yPred = lr.predict(transformed_X_test)
print('accuracy %s' % accuracy_score(yPred, transformed_y_test))
print(classification_report(transformed_y_test, yPred))

accuracy 0.875
              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.89      0.80      0.84        10
           2       0.83      0.94      0.88        16

    accuracy                           0.88        32
   macro avg       0.91      0.86      0.88        32
weighted avg       0.88      0.88      0.87        32

